In [ ]:
"""
tensor completion solver with split conic solver (SCS)
"""
from math_utils import *
import numpy as np
from scipy import linalg
from sktensor import ktensor
import matplotlib.pyplot as plt
import cPickle as pickle
import sys
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
import scs 
from sys import getrefcount
from guppy import hpy
from scipy import sparse

In [ ]:
def gen_mask(shape,obv_ratio):
    '''
    Return a mask matrix, 1 for observed, 0 for missing. 
    Args:
        obv_ratio: (0,1) observation ratio
        N: size of the mask
    '''
    np.random.seed()
    obv_idx = np.random.choice([0, 1], size=shape, p=[1.0-obv_ratio, obv_ratio])
    Omega = np.reshape(obv_idx, shape)
    return Omega 

In [ ]:
def gen_orth_tensor(N,R):
    # all orthogonal cases
    U = np.random.random((N, R))
    U_orth = linalg.orth(U)

    V = np.random.random((N, R))
    V_orth = linalg.orth(V)

    W = np.random.random((N, R))
    W_orth = linalg.orth(W)

    Lambda = np.random.random((R,))

    X = ktensor([U_orth, V_orth, W_orth], lmbda=Lambda)
    X_ten = np.asarray(X.totensor())

    return X_ten

In [ ]:
"""
reformulate tensor n-rank minimization as a SCS problem
"""
ij = np.array([[0,1,2,3],[0,1,2,3]])
A = sparse.csc_matrix(([-1.,-1.,1.,1.], ij), (4,4)) 
b = np.array([0.,0.,1,1])
c = np.array([1.,1.,-1,-1])
cone = {'l': 4}

print c
print b
print A
print cone
data = {'A': A, 'b':b, 'c':c}

sol = scs.solve(data, cone, use_indirect = False)
print sol

In [39]:
import sys
from numpy import linalg as LA
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
from cvxpy import *
from math_utils import *


def tensor_recovery(Omega,X, succ_thres):
    shape = X.shape
    n_elems = np.prod(shape)
    X_opt_1 = Variable(shape[0], n_elems / shape[0])
    X_opt_2 = Variable(shape[1], n_elems / shape[1])
    X_opt_3 = Variable(shape[2], n_elems / shape[2])

    # mask

    Omega_1 = unfold(Omega,0)
    Omega_2 = unfold(Omega,1)
    Omega_3 = unfold(Omega,2)

    # measurements
    X_1 = unfold(X,0)
    X_2 = unfold(X,1)
    X_3 = unfold(X,2)

    obj = Minimize(norm(X_opt_1, "nuc")+ norm(X_opt_2, "nuc") + norm(X_opt_3, "nuc") ) # tensor norm as algebraic mean of matrix norm
    constraints = [mul_elemwise(Omega_1, X_opt_1) == mul_elemwise(Omega_1, X_1),
                  mul_elemwise(Omega_2, X_opt_2) == mul_elemwise(Omega_2, X_2),
                  mul_elemwise(Omega_3, X_opt_3) == mul_elemwise(Omega_3, X_3)]
    #               vec(X_opt_1)== vec((vstack(X_opt_2[:,0:shape[0]],X_opt_2[:,shape[0]:2*shape[0]],
    #                                          X_opt_2[:,2*shape[0]:3*shape[0]], X_opt_2[:,3*shape[0]:4*shape[0]])).T)] # vector format of the variables are the same

    prob = Problem(obj, constraints)
    # Use SCS to solve the problem.
    prob.solve(verbose=True, solver=SCS) 
    X_out_1  = np.asarray(X_opt_1.value)
    X_out_2 = np.asarray(X_opt_2.value)
    X_out_3 = np.asarray(X_opt_3.value)

    X_out = fold(X_out_1,0,shape)#1./3* np.add(np.add(fold(X_out_1, 0, shape), fold(X_out_2, 1, shape) ), fold(X_out_3, 2,shape))
    err = tensor_norm(X_out - X, 'fro')/ tensor_norm(X, 'fro')
    print 'recovery err:', err
    if  err < succ_thres:
        return True
    else:
        return False

In [ ]:
# Setting for the experiments   
n_rows = 5;
n_cols = 5;
num_exp = 5
N = 50; # size
max_rank = N

# obv_ratio = 0.8;
# succ_thres = 1e-3;
ten_recv_prob = np.zeros((n_rows, n_cols))

dgr_range = np.linspace(0.1, 1, n_rows)
obv_range = np.linspace(0,1,n_cols)   

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

inputs = range(10) 
def ten_rand_exp(X, obv_ratio, exp_id):
    succ_thres = 1.0e-3
    N = len(X)
    Omega = gen_mask((N,N,N),obv_ratio)
    if tensor_recovery(Omega, X, succ_thres):
        return 1
    else:
        return 0
   
    
num_cores = multiprocessing.cpu_count()
"""degree of freedom funciton """
dgr_func = lambda n, d: np.ceil(n -  np.sqrt(n*n - n*n*d)) # n^2 number of measurements

for i in range(n_rows):
    dgr = dgr_range[i]
    R = dgr_func(N,dgr)
    X_ten = gen_orth_tensor (N,R)
    print "data generated"
    for j in range(n_cols):
        obv_ratio = obv_range[j]
        result = Parallel(n_jobs=num_cores)(delayed(ten_rand_exp)(X_ten,obv_ratio, exp_id) 
                                             for exp_id in range(num_exp))
        succ_rate = 1.0* np.sum(np.asarray(result))/num_exp
        print('rank', R , 'obv_ratio', obv_ratio, 'succ_rate', succ_rate)
        ten_recv_prob[i,j] = succ_rate
        
output_data ={'xticks':obv_range, 'yticks':dgr_range, 'values':ten_recv_prob}
pickle.dump( output_data, open( "result/ten_recv_50.pkl", "wb" ) )

In [40]:
N = 20
R = 1
obv_ratio = 0.8
X_ten = np.random.rand(N,N,N)
Omega = gen_mask((N,N,N),obv_ratio)
succ_thres = 1e-3
rslt = tensor_recovery(Omega, X_ten, succ_thres)

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 860298, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 553200, constraints m = 577200
Cones:	primal zero / dual free vars: 311970
	sd vars: 265230, sd blks: 3
Setup time: 5.88e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  4.16e-01 
   100| 1.72e-03  1.72e-03  8.49e-04  4.21e+02  4.21e+02  1.51e-13  9.98e+01 
   120| 9.54e-04  4.82e-04  2.05e-04  4.23e+02  4.23e+02  1.51e-13  1.18e+02 
----